# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## 1. Датасет и общая информация

In [1]:
import pandas as pd

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression



from sklearn.linear_model import LinearRegression

from sklearn.ensemble import RandomForestRegressor

In [2]:
df = pd.read_csv('/datasets/users_behavior.csv')

In [3]:
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


Проверка на мультиколлинеарность

In [5]:
df.corr()

,calls,minutes,messages,mb_used,is_ultra
calls,1.000000,0.982083,0.177385,0.286442,0.207122
minutes,0.982083,1.000000,0.173110,0.280967,0.206955
messages,0.177385,0.173110,1.000000,0.195721,0.203830
mb_used,0.286442,0.280967,0.195721,1.000000,0.198568
is_ultra,0.207122,0.206955,0.203830,0.198568,1.000000


<font color='blue'>
    
Проверка корреляции показала очень высокую прямую зависимость **calls** и **minutes**
    
Удалим признак **minutes**, т.к. **calls** демонстрирует более высокую корреляцию с целевым признаком **is_ultra**, а также с другими признаками
    
</font>

In [6]:
df = df.drop(['minutes'], axis=1)

In [7]:
df.head()

,calls,messages,mb_used,is_ultra
0,40.0,83.0,19915.42,0
1,85.0,56.0,22696.96,0
2,77.0,86.0,21060.45,0
3,106.0,81.0,8437.39,1
4,66.0,1.0,14502.75,0


## 2. Разбиение данных на выборки для обучения

Разделять датасет будем на обучающую и равные валидационную и тестовую.

Для этого данные предварительно разделим на обучающую 80% и валидационную выборку 20% - для получения равных валидационной и тестовой выборок

In [11]:
train_1, df_valid = train_test_split(df, test_size=0.2, random_state=42) 

Разделяем полученную выборку train_1 данные на обучающую 75% и тестовую выборки 25%.

In [12]:
df_train,df_test = train_test_split(train_1, test_size=0.25, random_state=42) 

Проверка длины разделенных таблиц

In [13]:
len(df) - len(df_train) - len(df_valid) - len(df_test)

0

In [14]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1928 entries, 3187 to 2477
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     1928 non-null   float64
 1   messages  1928 non-null   float64
 2   mb_used   1928 non-null   float64
 3   is_ultra  1928 non-null   int64  
dtypes: float64(3), int64(1)
memory usage: 75.3 KB


In [15]:
df_valid.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 643 entries, 506 to 22
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     643 non-null    float64
 1   messages  643 non-null    float64
 2   mb_used   643 non-null    float64
 3   is_ultra  643 non-null    int64  
dtypes: float64(3), int64(1)
memory usage: 25.1 KB


In [16]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 643 entries, 2644 to 1400
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     643 non-null    float64
 1   messages  643 non-null    float64
 2   mb_used   643 non-null    float64
 3   is_ultra  643 non-null    int64  
dtypes: float64(3), int64(1)
memory usage: 25.1 KB


Признаки обучающие

In [17]:
features_train = df_train.drop('is_ultra', axis = 1)

Целевой признак обучающий

In [18]:
target_train = df_train['is_ultra']

Признаки валидационные

In [19]:
features_valid = df_valid.drop('is_ultra', axis = 1)

Целевой признак валидационный

In [20]:
target_valid = df_valid['is_ultra']

Признаки тестовые

In [21]:
features_test = df_test.drop('is_ultra', axis = 1)

Целевой признак тестовый

In [22]:
target_test = df_test['is_ultra']

## 3. Исследование модели

Для обучения: features_train и target_train

Для проверки: features_valid и target_valid

features_test и target_test откладываем нетронутыми.

### 3.1 Модель - решающее дерево

Гиперпараметр max_depth - проверку проводим на валидационных выборках. 

In [23]:
%%time
best_model_DecisionTreeClassifier = None
best_result_DecisionTreeClassifier = 0
for depth in range(1, 40):
    
    # обучение модели с заданным количеством деревьев est
    model_DecisionTreeClassifier = DecisionTreeClassifier(random_state=42, max_depth=depth)
    
    # обучение модели на тренировочной выборке
    model_DecisionTreeClassifier.fit(features_train, target_train)
    
    # рассчет качества модели на валидационной выборке
    result_DecisionTreeClassifier = model_DecisionTreeClassifier.score(features_valid, target_valid) 
    if result_DecisionTreeClassifier > best_result_DecisionTreeClassifier:
        
        # сохраняем наилучшую модель
        best_model_DecisionTreeClassifier =  model_DecisionTreeClassifier
        
        #  сохраняем наилучшее значение метрики accuracy на валидационных данных
        best_result_DecisionTreeClassifier = result_DecisionTreeClassifier 

print("Accuracy наилучшей модели решающее дерево на валидационной выборке:", best_result_DecisionTreeClassifier)
print(best_model_DecisionTreeClassifier)

Accuracy наилучшей модели решающее дерево на валидационной выборке: 0.7931570762052877
DecisionTreeClassifier(max_depth=10, random_state=42)
CPU times: user 279 ms, sys: 10.4 ms, total: 290 ms
Wall time: 288 ms


Предсказание. Модель - решающее дерево

In [24]:
predictions_test_DecisionTreeClassifier = model_DecisionTreeClassifier.predict(features_test)

Значение depth = 5 дает лучший результат Accuracy = 0.80

### 3.2 Модель - случайный лес

Алгоритм обучает большое количество независимых друг от друга деревьев, а потом принимает решение на основе голосования. Случайный лес помогает улучшить результат предсказания и избежать переобучения.

n_estimator - количество деревьев, чем больше, тем точнее, но время обработки вырастает. 

In [25]:
%%time
best_model_RandomForestClassifier = None
best_result_RandomForestClassifier = 0
for est in range(1, 100):
    
    # обучение модели с заданным количеством деревьев est
    model_RandomForestClassifier = RandomForestClassifier(random_state=42, n_estimators=est)
    
    # обучение модели на тренировочной выборке
    model_RandomForestClassifier.fit(features_train, target_train)
    
    # рассчет качества модели на валидационной выборке
    result_RandomForestClassifier = model_RandomForestClassifier.score(features_valid, target_valid) 
    if result_RandomForestClassifier > best_result_RandomForestClassifier:
        
        # сохраняем наилучшую модель
        best_model_RandomForestClassifier =  model_RandomForestClassifier
        
        #  сохраняем наилучшее значение метрики accuracy на валидационных данных
        best_result_RandomForestClassifier = result_RandomForestClassifier 

print("Accuracy наилучшей модели случайный лес на валидационной выборке:", best_result_RandomForestClassifier)
print(best_model_RandomForestClassifier)

Accuracy наилучшей модели случайный лес на валидационной выборке: 0.807153965785381
RandomForestClassifier(n_estimators=31, random_state=42)
CPU times: user 13.8 s, sys: 86.5 ms, total: 13.9 s
Wall time: 14 s


Предсказание. Модель - случайный лес

In [26]:
predictions_test_RandomForestClassifier = model_RandomForestClassifier.predict(features_test)

Определено, что значение n_estimators = 50 дает наилучшее значение Accuracy = 0.81

### 3.3 Модель - логистическая регрессия

In [27]:
%%time
# инициализация модели логистической регрессии с параметром random_state=42
model_LogisticRegression  = LogisticRegression(random_state=42)

# обучение модели на тренировочной выборке
model_LogisticRegression.fit(features_train, target_train)

# получение метрики качества модели на валидационной выборке
result_LogisticRegression = model_LogisticRegression.score(features_valid, target_valid) 

print("Accuracy модели логистической регрессии на валидационной выборке:", result_LogisticRegression)

Accuracy модели логистической регрессии на валидационной выборке: 0.71850699844479
CPU times: user 17.3 ms, sys: 47 µs, total: 17.3 ms
Wall time: 18.9 ms


Предсказание. Модель - логистическая регрессия

In [28]:
predictions_test_LogisticRegression = model_LogisticRegression.predict(features_test)

Модель логистической регрессии дает значение Accuracy = 0.70. Модель быстрая, но неточная

### Вывод:

1. Качество (accuracy). 
Самое высокое качество у  решающего дереваа.
На втором месте — дерево решений.
Самое низкое качество предсказания у логистической регрессии.
2. Скорость работы.
Высокая скорость работы у логистической регрессии: у неё меньше всего параметров.
Скорость решающего дерева тоже высокая и зависит от глубины. 
Случайный лес медленнее всех.

**Модель решающего дерева**: скорость достаточно высокая и по качеству не значительно уступает модели  случайного леса.

## 4. Проверка модели на тестовой выборке

## Проверка корректности моделей метриками классификации

### Accuracy

#### Accuracy. Модель - решающее дерево

In [29]:
accuracy_score(predictions_test_DecisionTreeClassifier, target_test) 

0.7387247278382582

#### Accuracy. Модель - случайный лес

In [30]:
accuracy_score(predictions_test_RandomForestClassifier, target_test) 

0.7947122861586314

#### Accuracy. Модель - логистическая регрессия

In [31]:
accuracy_score(predictions_test_LogisticRegression, target_test) 

0.7247278382581649

### Recall

Полнота - доля положительных ответов среди всех.
Полнота — это доля TP-ответов среди всех, у которых истинная метка 1. Хорошо, когда значение recall близко к единице: модель хорошо ищет положительные объекты. Если ближе к нулю — модель надо перепроверить и починить.

#### Recall. Модель - решающее дерево

In [32]:
recall_score(predictions_test_DecisionTreeClassifier, target_test)

0.5666666666666667

#### Recall. Модель - случайный лес

In [33]:
accuracy_score(predictions_test_RandomForestClassifier, target_test) 

0.7947122861586314

#### Recall. Модель - логистическая регрессия

In [34]:
recall_score(predictions_test_LogisticRegression, target_test)

0.7142857142857143

### Precision

Точность - как много отрицательных ответов нашла модель, пока искала положительные. Нам нужна точность, близкая к единице.

#### Precision. Модель - решающее дерево

In [35]:
precision_score(predictions_test_DecisionTreeClassifier, target_test)

0.53125

#### Precision. Модель - случайный лес

In [36]:
precision_score(predictions_test_RandomForestClassifier, target_test) 

0.5260416666666666

#### Precision. Модель - логистическая регрессия

In [37]:
precision_score(predictions_test_LogisticRegression, target_test)

0.13020833333333334

f1_score - это среднее гармоническое полноты и точности. Единица в F1 означает, что соотношение полноты и точности равно 1:1.

In [38]:
f1_score(predictions_test_DecisionTreeClassifier, target_test)

0.5483870967741935

In [39]:
f1_score(predictions_test_RandomForestClassifier, target_test)

0.6047904191616766

In [40]:
f1_score(predictions_test_LogisticRegression, target_test)

0.22026431718061673

### Вывод:

1. Качество

Метрики классификации accuracy, precision, recall показали разные значения моделей обучения.

Агрегирующая метрика f1_score указывает, что модель случайный лес имеет наивысшую точность

2. Скорость работы.

Метод подсчета скорости работы ячейки %%time показал самую высокую скорость у модели логистичесая

регрессия, а случайный лес имеет самое большое время выполнения. 

**Модель случайный лес**: наиболее предпочтительна из-за самого высокого качества, несмотря на большое

(более 12 сек) время обучения модели - не критичное в данном случае.

## 5. (бонус) Проверьте модели на адекватность

<font color='green'>Проверка на адекватность позволяет понять, хороша ли выбранная нами лучшая модель, или даже нет смысла с ней работать.\
Для этого принято использовать константную модель, которая всегда предсказывает одно и то же, или рандомную. Затем сравнивается ключевая метрика. Если наша лучшая модель показала метрику лучше, чем константная или рандомная, то проверка на адекватность пройдена\
Можно взять готовую <a href="https://scikit-learn.org/stable/modules/generated/sklearn.dummy.DummyClassifier.html">DummyClassifier</a>\
     Используется так же, как и другие модели sklearn: fit на обучающей выборке, predict на тестовой

<font color='green'>
Accuracy, F1, и ROC-AUC - отличные метрики, но нужно держать в голове, что это лишь обобщённые характеристики. А в реальной практике бизнесу часто важно знать детали самой ошибки - из каких ошибок/успехов она складывается. И в этом случае очень кстати будут меры полноты и точности, которые рассмотрены в этом проекте. Только через них можно выйти на финансовые возможности модели. Ведь перед запуском её в бой, будет сравнение стоимости создания и поддержки модели со стомостью "оставить как есть". \
Тем не менее ключевая метрика дана нам не просто так.\
В ДС всегда важны эксперименты</font>